In [ ]:
import numpy as np 
import pandas as pd
from datetime import timedelta
import os

In [4]:
CLIMATE_DATA_FILE = "Data/climate_data.csv"
VISITATION_DATA_FILE = "Data/visitation_data.csv"

# **0. Load and Preview the Given Datasets (Climate and Visitation)**

In [3]:
climate = pd.read_csv(CLIMATE_DATA_FILE)
visitation = pd.read_csv(VISITATION_DATA_FILE)

print("=== Climate Data ===")
display(climate.head())
print(climate.info())

print("\n=== Visitation Data ===")
display(visitation.head())
print(visitation.info())

=== Climate Data ===


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Bureau of Meteorology station number,Year,Month,Day,Maximum temperature (Degree C),Minimum temperature (Degree C),Rainfall amount (millimetres)
0,71075,2010,1,1,NaN,NaN,NaN
1,71075,2010,1,2,NaN,NaN,NaN
2,71075,2010,1,3,NaN,NaN,NaN
3,71075,2010,1,4,NaN,NaN,NaN
4,71075,2010,1,5,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39813 entries, 0 to 39812
Data columns (total 7 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Bureau of Meteorology station number  39813 non-null  int64  
 1   Year                                  39813 non-null  int64  
 2   Month                                 39813 non-null  int64  
 3   Day                                   39813 non-null  int64  
 4   Maximum temperature (Degree C)        38275 non-null  float64
 5   Minimum temperature (Degree C)        38280 non-null  float64
 6   Rainfall amount (millimetres)         37857 non-null  float64
dtypes: float64(3), int64(4)
memory usage: 2.1 MB
None

=== Visitation Data ===


,Year,Week,Mt. Baw Baw,Mt. Stirling,Mt. Hotham,Falls Creek,Mt. Buller,Selwyn,Thredbo,Perisher,Charlotte Pass
0,2014.0,1.0,555.0,60.0,3483.0,2790.0,8296.0,1041.0,5535.0,7370.0,408.0
1,2014.0,2.0,804.0,42.0,1253.0,1425.0,1987.0,383.0,2090.0,2751.0,151.0
2,2014.0,3.0,993.0,30.0,2992.0,2101.0,2413.0,597.0,3216.0,4255.0,230.0
3,2014.0,4.0,2976.0,165.0,9680.0,9544.0,18831.0,2877.0,15497.0,20265.0,1134.0
4,2014.0,5.0,11112.0,645.0,29628.0,26211.0,49217.0,8588.0,46546.0,61339.0,3403.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Year            165 non-null    float64
 1   Week            165 non-null    float64
 2   Mt. Baw Baw     165 non-null    float64
 3   Mt. Stirling    165 non-null    float64
 4   Mt. Hotham      165 non-null    float64
 5   Falls Creek     165 non-null    float64
 6   Mt. Buller      165 non-null    float64
 7   Selwyn          165 non-null    float64
 8   Thredbo         165 non-null    float64
 9   Perisher        165 non-null    float64
 10  Charlotte Pass  165 non-null    float64
dtypes: float64(11)
memory usage: 86.0 KB
None



# Notes:
### 1. missing data
### 2. combine two datasets

### Drop all 100% empty rows: 

In [4]:
print("Climate rows:", climate.shape[0])
print("Visitation rows:", visitation.shape[0])

Climate rows: 39813
Visitation rows: 999


In [5]:
climate = climate.dropna(how="all").copy()
visitation = visitation.dropna(how="all").copy()

In [6]:
print("After dropping empties:")
print("Climate rows:", climate.shape[0])
print("Visitation rows:", visitation.shape[0])

After dropping empties:
Climate rows: 39813
Visitation rows: 165


# **1. Build Date Cols and Match Station to Resort (excluding Charlotte Pass)**

In [ ]:
STATION_COL = "Bureau of Meteorology station number"

# check Date exists
if "Date" not in climate.columns:
    climate["Date"] = pd.to_datetime(
        dict(
            year=climate["Year"].astype("Int64"),
            month=climate["Month"].astype("Int64"),
            day=climate["Day"].astype("Int64"),
        ),
        errors="coerce",
    )

# 2) Resort <=> Station mapping (Charlotte Pass handled later)
resort_to_station = {
    "Mt. Baw Baw": [85291],
    "Mt. Stirling": [83024],
    "Mt. Hotham": [83085],
    "Falls Creek": [83084],
    "Mt. Buller": [83024],
    "Selwyn": [72161],
    "Thredbo": [71032],
    "Perisher": [71075],
    "Charlotte Pass": [71032, 71075],  # do later
}

# station -> resort not inc Charlotte Pass
station_to_resort = {
    s: r for r, ss in resort_to_station.items() if r != "Charlotte Pass" for s in ss
}

# Add Resort column
climate["Resort"] = climate[STATION_COL].map(station_to_resort)

# preview
cols = [c for c in ["Date", STATION_COL, "Resort"] if c in climate.columns]
display(climate[cols].head(10))

# sanity check counts by resort
print(climate["Resort"].value_counts(dropna=False))


,Date,Bureau of Meteorology station number,Resort
0,2010-01-01,71075,Perisher
1,2010-01-02,71075,Perisher
2,2010-01-03,71075,Perisher
3,2010-01-04,71075,Perisher
4,2010-01-05,71075,Perisher
5,2010-01-06,71075,Perisher
6,2010-01-07,71075,Perisher
7,2010-01-08,71075,Perisher
8,2010-01-09,71075,Perisher
9,2010-01-10,71075,Perisher


Resort
Perisher       5688
Mt. Buller     5688
Falls Creek    5688
Mt. Hotham     5688
Mt. Baw Baw    5688
Selwyn         5688
Thredbo        5685
Name: count, dtype: int64


In [8]:
print("Climate rows:", climate.shape[0])
print("Visitation rows:", visitation.shape[0])

Climate rows: 39813
Visitation rows: 165


# **Step 2: Charlotte Pass = average of 71032 & 71075 (by Date)**

In [9]:
STATION_COL = "Bureau of Meteorology station number"
MAX_COL = "Maximum temperature (Degree C)"
MIN_COL = "Minimum temperature (Degree C)"
RAIN_COL = "Rainfall amount (millimetres)"

# take only the two stations, group by Date, average numeric climate vars
cp = (climate[climate[STATION_COL].isin([71032, 71075])]
        .groupby("Date", as_index=False)[[MAX_COL, MIN_COL, RAIN_COL]]
        .mean())

cp["Resort"] = "Charlotte Pass"
cp[STATION_COL] = np.nan  # no single station; it's an average

# append to climate
climate = pd.concat([climate, cp], ignore_index=True)

# quick sanity check
print("Added Charlotte Pass rows:", (climate["Resort"] == "Charlotte Pass").sum())
display(climate.loc[climate["Resort"]=="Charlotte Pass",
                    ["Date","Resort", MAX_COL, MIN_COL, RAIN_COL]].head())


Added Charlotte Pass rows: 5688


,Date,Resort,Maximum temperature (Degree C),Minimum temperature (Degree C),Rainfall amount (millimetres)
39813,2010-01-01,Charlotte Pass,17.6,9.4,47.2
39814,2010-01-02,Charlotte Pass,13.8,9.3,29.2
39815,2010-01-03,Charlotte Pass,14.3,6.3,0.0
39816,2010-01-04,Charlotte Pass,14.9,5.1,0.0
39817,2010-01-05,Charlotte Pass,17.7,10.4,0.0


In [10]:
climate

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Bureau of Meteorology station number,Year,Month,Day,Maximum temperature (Degree C),Minimum temperature (Degree C),Rainfall amount (millimetres),Date,Resort
0,71075.0,2010.0,1.0,1.0,NaN,NaN,NaN,2010-01-01,Perisher
1,71075.0,2010.0,1.0,2.0,NaN,NaN,NaN,2010-01-02,Perisher
2,71075.0,2010.0,1.0,3.0,NaN,NaN,NaN,2010-01-03,Perisher
3,71075.0,2010.0,1.0,4.0,NaN,NaN,NaN,2010-01-04,Perisher
4,71075.0,2010.0,1.0,5.0,NaN,NaN,NaN,2010-01-05,Perisher
...,...,...,...,...,...,...,...,...,...
45496,NaN,NaN,NaN,NaN,-1.9,-5.45,3.8,2025-07-24,Charlotte Pass
45497,NaN,NaN,NaN,NaN,5.8,-9.15,0.0,2025-07-25,Charlotte Pass
45498,NaN,NaN,NaN,NaN,1.4,-3.20,8.8,2025-07-26,Charlotte Pass
45499,NaN,NaN,NaN,NaN,1.4,0.10,36.4,2025-07-27,Charlotte Pass


In [11]:
# remove the useless columns and rearrange the remaining columns
keep_cols = ["Date", "Resort",
             "Maximum temperature (Degree C)",
             "Minimum temperature (Degree C)",
             "Rainfall amount (millimetres)"]

climate = climate[keep_cols].copy()

display(climate.iloc[2000:2100])

,Date,Resort,Maximum temperature (Degree C),Minimum temperature (Degree C),Rainfall amount (millimetres)
2000,2015-06-24,Perisher,5.1,0.9,0.0
2001,2015-06-25,Perisher,8.0,-0.5,0.0
2002,2015-06-26,Perisher,10.4,-6.1,0.0
2003,2015-06-27,Perisher,12.5,-5.9,0.0
2004,2015-06-28,Perisher,11.4,-4.4,0.0
...,...,...,...,...,...
2095,2015-09-27,Perisher,11.5,-3.5,0.0
2096,2015-09-28,Perisher,9.5,-2.0,0.0
2097,2015-09-29,Perisher,8.2,3.4,0.0
2098,2015-09-30,Perisher,9.9,0.8,0.0


In [12]:
print("Climate rows:", climate.shape[0])
print("Visitation rows:", visitation.shape[0])

Climate rows: 45501
Visitation rows: 165


# **3. Convert Days to Weeks**
### **NB: (only ski season weeks are kept)**

In [ ]:
# check Date exists and is datetime
clim = climate.copy()
clim["Date"] = pd.to_datetime(clim["Date"], errors="coerce")

# Derive years from Date (from not missing 'Year')
years = sorted(clim["Date"].dt.year.dropna().astype(int).unique())

md_starts = ["06-09","06-16","06-23","06-30","07-07","07-14","07-21","07-28",
             "08-04","08-11","08-18","08-25","09-01","09-08","09-15"]

rows = []
for y in years:
    for i, md in enumerate(md_starts, start=1):
        s = pd.to_datetime(f"{y}-{md}")
        rows.append({"Year": y, "SkiWeek": i, "WeekStart": s, "WeekEnd": s + timedelta(days=7)})

weeks = pd.DataFrame(rows).astype({"Year":"int64","SkiWeek":"int64"})
weeks["WeekStart"] = pd.to_datetime(weeks["WeekStart"])


In [ ]:
# ensure datetime dtypes
clim = climate.dropna(subset=["Resort"]).copy()
clim["Date"] = pd.to_datetime(clim["Date"])
weeks["WeekStart"] = pd.to_datetime(weeks["WeekStart"])

# ensure SAME int dtype for the 'by' key
clim["Year"]  = clim["Date"].dt.year.astype("int64")
weeks["Year"] = weeks["Year"].astype("int64")

# (re)compute WeekEnd in case it's missing
weeks["WeekEnd"] = weeks["WeekStart"] + pd.Timedelta(days=7)

# sort as required by merge_asof
clim  = clim.sort_values(["Year", "Date"])
weeks = weeks.sort_values(["Year", "WeekStart"])

# merge_asof: assign the latest WeekStart <= Date within the same Year
clim = pd.merge_asof(
    clim, weeks,
    left_on="Date", right_on="WeekStart",
    by="Year", direction="backward", allow_exact_matches=True
)

# keep only dates inside the 7-day window
clim = clim[clim["Date"] < clim["WeekEnd"]].copy()

# aggregate to ski-season weekly per resort
MAX_COL, MIN_COL, RAIN_COL = (
    "Maximum temperature (Degree C)",
    "Minimum temperature (Degree C)",
    "Rainfall amount (millimetres)",
)

climate_weekly = (clim
    .groupby(["Year","SkiWeek","Resort"], as_index=False)
    .agg({MAX_COL:"mean", MIN_COL:"mean", RAIN_COL:"sum"})
    .sort_values(["Year","SkiWeek","Resort"])
)

print("Dataset adjustment done:", climate_weekly.shape)
climate_weekly.head()

Dataset adjustment done: (1863, 6)


,Year,SkiWeek,Resort,Maximum temperature (Degree C),Minimum temperature (Degree C),Rainfall amount (millimetres)
0,2010,1.0,Charlotte Pass,1.528571,-6.078571,7.2
1,2010,1.0,Falls Creek,0.500000,-4.657143,3.6
2,2010,1.0,Mt. Baw Baw,0.071429,-2.557143,39.6
3,2010,1.0,Mt. Buller,0.360000,-3.585714,25.4
4,2010,1.0,Mt. Hotham,-1.114286,-5.057143,8.0


In [15]:
print("Climate rows:", climate.shape[0])
print("Climate_weekly rows:", climate_weekly.shape[0])
print("Visitation rows:", visitation.shape[0])

Climate rows: 45501
Climate_weekly rows: 1863
Visitation rows: 165


# **4. Deal with Missing Data**

In [16]:
# Total rows
n_total = len(visitation)

# Count + percentage of missing
for col in ["Year", "Week"]:
    n_missing = visitation[col].isna().sum()
    pct_missing = (n_missing / n_total) * 100
    print(f"{col}: {n_missing} missing ({pct_missing:.2f}% of rows)")

# Combined check
n_rows_missing = visitation[["Year","Week"]].isna().any(axis=1).sum()
pct_rows_missing = (n_rows_missing / n_total) * 100
print(f"\nRows with missing Year or Week: {n_rows_missing} "
      f"({pct_rows_missing:.2f}% of dataset)")

# Preview some of those rows
display(visitation[visitation[["Year","Week"]].isna().any(axis=1)].head(10))


Year: 0 missing (0.00% of rows)
Week: 0 missing (0.00% of rows)

Rows with missing Year or Week: 0 (0.00% of dataset)


,Year,Week,Mt. Baw Baw,Mt. Stirling,Mt. Hotham,Falls Creek,Mt. Buller,Selwyn,Thredbo,Perisher,Charlotte Pass


In [17]:
# total rows
n_total = len(climate_weekly)

# rows with at least one missing value
n_missing_rows = climate_weekly.isna().any(axis=1).sum()

# percentage
pct_missing_rows = (n_missing_rows / n_total) * 100

print(f"Rows with missing values: {n_missing_rows} / {n_total} "
      f"({pct_missing_rows:.2f}%)")

# optional: see which columns are most often missing
missing_by_col = climate_weekly.isna().mean() * 100
print("\nPercentage missing by column:")
print(missing_by_col)

Rows with missing values: 32 / 1863 (1.72%)

Percentage missing by column:
Year                              0.000000
SkiWeek                           0.000000
Resort                            0.000000
Maximum temperature (Degree C)    1.610306
Minimum temperature (Degree C)    1.610306
Rainfall amount (millimetres)     0.000000
dtype: float64


### **Impute missing temps by resort + ski week average (seasonal and location-specific)**

In [ ]:
# Columns to impute
cols_to_impute = ["Maximum temperature (Degree C)", "Minimum temperature (Degree C)"]

for col in cols_to_impute:
    climate_weekly[col] = (
        climate_weekly.groupby(["Resort", "SkiWeek"])[col]
        .transform(lambda x: x.fillna(x.mean()))
    )
    # If still NaN (e.g., whole group missing), fill with overall mean
    climate_weekly[col] = climate_weekly[col].fillna(climate_weekly[col].mean())

print("Remaining missing values per column:")
print(climate_weekly[cols_to_impute].isna().sum())

Remaining missing values per column:
Maximum temperature (Degree C)    0
Minimum temperature (Degree C)    0
dtype: int64


In [19]:
climate_weekly

,Year,SkiWeek,Resort,Maximum temperature (Degree C),Minimum temperature (Degree C),Rainfall amount (millimetres)
0,2010,1.0,Charlotte Pass,1.528571,-6.078571,7.2
1,2010,1.0,Falls Creek,0.500000,-4.657143,3.6
2,2010,1.0,Mt. Baw Baw,0.071429,-2.557143,39.6
3,2010,1.0,Mt. Buller,0.360000,-3.585714,25.4
4,2010,1.0,Mt. Hotham,-1.114286,-5.057143,8.0
...,...,...,...,...,...,...
1858,2025,8.0,Mt. Baw Baw,0.900000,0.000000,15.6
1859,2025,8.0,Mt. Buller,0.300000,-1.200000,38.0
1860,2025,8.0,Mt. Hotham,-0.100000,-1.700000,32.8
1861,2025,8.0,Perisher,0.900000,-1.100000,49.0


# **5. Reshape Visitation and Merge with Climate Data**

In [20]:
# 1) Clean & relabel
vis = visitation.dropna(how="all").dropna(subset=["Year","Week"]).copy()

# Drop rows where all resort counts are 0
resort_cols = [c for c in visitation.columns if c not in ["Year","Week"]]
visitation = visitation[~(visitation[resort_cols].sum(axis=1) == 0)].copy()

### **NB: There are some rows in visitation that contain only 0, so drop**

In [ ]:
vis["Year"] = vis["Year"].astype(int)
vis["SkiWeek"] = vis["Week"].astype(int)

# Wide to long: Year, SkiWeek, Resort, Visitors
vis_long = vis.melt(
    id_vars=["Year","SkiWeek"],
    var_name="Resort",
    value_name="Visitors"
).dropna(subset=["Visitors"])

vis_long["Visitors"] = vis_long["Visitors"].astype(int)

print("vis_long shape:", vis_long.shape)
display(vis_long.head())

# Merge with climate
merged = vis_long.merge(climate_weekly, on=["Year","SkiWeek","Resort"], how="inner")
print("merged shape:", merged.shape)
display(merged.head())

vis_long shape: (1650, 4)


,Year,SkiWeek,Resort,Visitors
0,2014,1,Week,1
1,2014,2,Week,2
2,2014,3,Week,3
3,2014,4,Week,4
4,2014,5,Week,5


merged shape: (1320, 7)


,Year,SkiWeek,Resort,Visitors,Maximum temperature (Degree C),Minimum temperature (Degree C),Rainfall amount (millimetres)
0,2014,1,Mt. Baw Baw,555,4.957143,1.100000,24.4
1,2014,2,Mt. Baw Baw,804,5.485714,0.814286,21.0
2,2014,3,Mt. Baw Baw,993,1.042857,-1.814286,76.4
3,2014,4,Mt. Baw Baw,2976,2.528571,-0.957143,82.0
4,2014,5,Mt. Baw Baw,11112,0.685714,-2.257143,82.6


### **Drop all rows that has 0 visotor**

In [ ]:
# Check
zero_vis_long = (vis_long["Visitors"] == 0).sum()
print(f"[vis_long] 0-visitor rows: {zero_vis_long} / {len(vis_long)} ({zero_vis_long/len(vis_long)*100:.2f}%)")

zero_merged = (merged["Visitors"] == 0).sum()
print(f"[merged]  0-visitor rows: {zero_merged} / {len(merged)} ({zero_merged/len(merged)*100:.2f}%)")

# Drop rows with 0 visitors 
vis_long = vis_long[vis_long["Visitors"] != 0].copy()
merged = merged[merged["Visitors"] != 0].copy()

print("\nAfter dropping 0-visitor rows:")
print("vis_long shape:", vis_long.shape)
print("merged shape:", merged.shape)

[vis_long] 0-visitor rows: 66 / 1650 (4.00%)
[merged]  0-visitor rows: 53 / 1320 (4.02%)

After dropping 0-visitor rows:
vis_long shape: (1584, 4)
merged shape: (1267, 7)


# **6. Check Sanity/ Verify Coverage**

In [ ]:
# sanity check
print("Resorts:", vis_long["Resort"].nunique())
print("Years:", vis_long["Year"].min(), "→", vis_long["Year"].max())
print("Weeks per year (visitation):")
print(vis_long.groupby("Year")["SkiWeek"].nunique())

# Coverage of merge
k = ["Year","SkiWeek","Resort"]
tot = len(vis_long)
ok  = len(merged)
print(f"Merge coverage: {ok}/{tot} ({ok/tot*100:.1f}%)")

# missing after merge (anti-join)
missing = (
    vis_long.merge(climate_weekly[k], on=k, how="left", indicator=True)
            .loc[lambda d: d["_merge"]=="left_only", k]
)
print("Missing combos:", missing.shape[0])
display(missing.head(10))

# Where gaps concentrate
gap_summary = (missing.value_counts(["Resort","Year"])
                        .rename("missing_weeks")
                        .reset_index()
                        .sort_values(["Resort","Year"]))
display(gap_summary.head(20))


Resorts: 10
Years: 2014 → 2024
Weeks per year (visitation):
Year
2014    15
2015    15
2016    15
2017    15
2018    15
2019    15
2020    15
2021    15
2022    15
2023    15
2024    15
Name: SkiWeek, dtype: int64
Merge coverage: 1267/1584 (80.0%)
Missing combos: 317


,Year,SkiWeek,Resort
0,2014,1,Week
1,2014,2,Week
2,2014,3,Week
3,2014,4,Week
4,2014,5,Week
5,2014,6,Week
6,2014,7,Week
7,2014,8,Week
8,2014,9,Week
9,2014,10,Week


,Resort,Year,missing_weeks
0,Mt. Stirling,2014,15
1,Mt. Stirling,2015,15
16,Mt. Stirling,2016,15
15,Mt. Stirling,2017,15
14,Mt. Stirling,2018,15
13,Mt. Stirling,2019,15
21,Mt. Stirling,2020,10
20,Mt. Stirling,2021,12
12,Mt. Stirling,2022,15
18,Mt. Stirling,2023,13


### **Take notes down**
Resorts: 10 → matches visitation file (Baw Baw, Stirling, Hotham, Falls, Buller, Selwyn, Thredbo, Perisher, Charlotte Pass, plus that stray “Week” column).

Years: 2014 → 2024 → full ski-season range in visitation.

Weeks per year: always 15 → good, matches ski-season definition.

Merge coverage: 1320 / 1650 (80%) → means climate data covers ~80% of resort-weeks. The missing 20% are expected when some resorts don’t have climate stations (or were mistyped).

Missing combos (330):

Mt. Stirling is missing entirely (all 15 weeks × all years) → that’s expected, because in your mapping Mt. Stirling shares a station code with Mt. Buller, and looks like we didn’t assign it separately.

The extra "Week" “resort” is just an artifact from melting — we should drop that column before melting, since it’s not a real resort.

# **7. Output Final Processed Dataset**

In [24]:
print("Final dataset shape:", merged.shape)

# Set pandas display format for floats
pd.options.display.float_format = "{:.2f}".format

display(merged.head(20))

Final dataset shape: (1267, 7)


,Year,SkiWeek,Resort,Visitors,Maximum temperature (Degree C),Minimum temperature (Degree C),Rainfall amount (millimetres)
0,2014,1,Mt. Baw Baw,555,4.96,1.10,24.40
1,2014,2,Mt. Baw Baw,804,5.49,0.81,21.00
2,2014,3,Mt. Baw Baw,993,1.04,-1.81,76.40
3,2014,4,Mt. Baw Baw,2976,2.53,-0.96,82.00
4,2014,5,Mt. Baw Baw,11112,0.69,-2.26,82.60
5,2014,6,Mt. Baw Baw,13381,1.01,-2.43,57.60
6,2014,7,Mt. Baw Baw,8832,3.79,0.01,8.40
7,2014,8,Mt. Baw Baw,9347,1.70,-2.17,56.00
8,2014,9,Mt. Baw Baw,10071,2.11,-2.01,11.60
9,2014,10,Mt. Baw Baw,11071,3.54,-2.11,23.20


In [25]:
merged.to_csv("final_dataset.csv", index=False)
print("Saved final dataset to final_dataset.csv")

Saved final dataset to final_dataset.csv
